In [2]:
import os
import sys
import random
import re
import warnings
import pandas as pd
import SimpleITK as sitk
from random import shuffle

In [22]:
#
outdata_dir = 'dataset_info'

In [44]:
re.findall("\d+","asd1234")

['1234']

In [52]:
# CT Data preprocess 
# To get the train, validation and test txt. 
# The data from same patient ID should be split into same group.

# The data are split conserved in different directory based on the labels.
warnings.filterwarnings('ignore')
root = '/media/tx-deepocean/Data/2022/chongfu1/Dataset/20220331'
neg_dir_list = ['R0']
pos_dir_list = ['R1', 'R2']
img_dir_name = 'IMG'
mask_dir_name = 'ANNO'
col_names = ['pid', 'pid1', 'label', 'RD', 'img_type', 'image', 'mask', 'series', 'image_size', 'mask_size', 'size_consis']

label = 0
# columns = ['Patients ID1', 'Patients ID2, 'label', 'sequence ID', 'image path', 'mask path',  
#            'Layer number', 'label number', 'Series description', 'Series date']
# def pair_image_mask(root, img_dir='IMG', mask_dir='ANNO'):


series_num = lambda mask_path: int(re.findall("\d+",mask_path.split('/')[-1].split('.')[0])[0])%10
def pair_image_mask(root, data_dir_list, label):
    datas = []
    for dir_name in data_dir_list:
        data_dir = os.path.join(root,dir_name)
        for pid_dir in os.listdir(data_dir):
            # Extract the series number from string.
            pid = re.findall(r'\d+', pid_dir)
            pid_path = os.path.join(data_dir, pid_dir)
            for il_dir in os.listdir(pid_path):
                tag = il_dir.split('_')[0]
                if tag == 'img':
                    img_dir = os.path.join(pid_path,il_dir)
                elif tag == 'label':
                    mask_dir = os.path.join(pid_path, il_dir)
            img_path_list = []
            mask_path_list = []
            for img in os.listdir(img_dir):
                img_path_list.append(os.path.join(img_dir, img))
            for mask in os.listdir(mask_dir):
                mask_path_list.append(os.path.join(mask_dir, mask))
            img_mask_pair = pair_im(img_path_list, mask_path_list)
            for pair in img_mask_pair:
                RD = dir_name
                img_size, mask_size = read_size(pair[0], pair[1])
                size_consis = (img_size == mask_size)
                img_path, img_type = exact_imgpath(pair[0])
                datas.append([pid[0], pid[1],label, RD, img_type, img_path, pair[1], series_num(pair[1]), img_size, mask_size, size_consis])
                              #pair[1].replace(root,''), series_num(pair[1]), img_size, mask_size, size_consis])
    return datas

def ex_pair_image_mask(root, data_dir_list, label):
    datas = []
    for dir_name in data_dir_list:
        data_dir = os.path.join(root,dir_name)
        for pid_dir in os.listdir(data_dir):
            # Extract the series number from string.
            pid = re.findall(r'\d+', pid_dir)
            pid_path = os.path.join(data_dir, pid_dir)
            for il_dir in os.listdir(pid_path):
                tag = il_dir.split('_')[0]
                if tag == 'img':
                    img_dir = os.path.join(pid_path,il_dir)
                elif tag == 'label':
                    mask_dir = os.path.join(pid_path, il_dir)
            img_path_list = []
            mask_path_list = []
            for img in os.listdir(img_dir):
                img_path_list.append(os.path.join(img_dir, img))
            for mask in os.listdir(mask_dir):
                mask_path_list.append(os.path.join(mask_dir, mask))
            img_mask_pair = pair_im(img_path_list, mask_path_list)
            for pair in img_mask_pair:
                RD = dir_name
                # img_size, mask_size = read_size(pair[0], pair[1])
                img_size, mask_size = '(256,256,30)', '(256,256,30)'
                size_consis = (img_size == mask_size)
                img_path, img_type = exact_imgpath(pair[0])
                datas.append([pid[0], pid[1],label, RD, img_type, img_path, pair[1], series_num(pair[1]), img_size, mask_size, size_consis])
                              #pair[1].replace(root,''), series_num(pair[1]), img_size, mask_size, size_consis])
    return datas

def pair_im(img_paths, mask_paths):
    for path in img_paths:
        if not os.path.exists(path):
            sys.stderr.write('No file exists!!!\n')
            raise SystemExit(1)
    for path in mask_paths:
        if not os.path.exists(path):
            sys.stderr.write('No file exists!!!\n')
            raise SystemExit(1)
    img_dic = {img.split('/')[-1].split('.')[0]:img for img in img_paths}
    mask_dic = {mask.split('/')[-1].split('.')[0]:mask for mask in mask_paths}
    keys = img_dic.keys()
    pairs = [[img_dic[key], mask_dic[key]] for key in keys]
    return pairs

def read_size(img_path, mask_path):
    num_files = len(os.listdir(img_path))
    if(num_files > 1):
        reader = sitk.ImageSeriesReader()
        dicomName = reader.GetGDCMSeriesFileNames(img_path)
        reader.SetFileNames(dicomName)
        sitkImage = reader.Execute()
    else:
        nii_path = os.path.join(img_path, os.listdir(img_path)[0])
        sitkImage = sitk.ReadImage(nii_path)
    img_size = sitkImage.GetSize()
    sitkMask = sitk.ReadImage(mask_path)
    mask_size = sitkMask.GetSize()
    return img_size, mask_size 

# If img is *.nii the return img_path/*.nii
# If img is dicom sequence the return img_path
def exact_imgpath(img_path):
    num_files = len(os.listdir(img_path))
    if(num_files > 1):
        return img_path, 'dicom'
    else:
        return os.path.join(img_path, os.listdir(img_path)[0]), 'nii'

In [51]:
icc_dir_list = ['ICC']
icc_data = pair_image_mask(root, icc_dir_list, label=2)
dficc = pd.DataFrame(icc_data, columns=col_names, index=None)
dficc[['pid']] = dficc[['pid']].astype(int)
dficc = dficc.sort_values(by=['pid','series'], ascending=(True, True))
dficc.to_csv('./dataset_info/icc_data_info.csv', sep=',', index=0)    
print('Total {} icc data'.format(dficc.shape[0]/3))

ImageSeriesReader (0x55b0a0727720): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000764127

ImageSeriesReader (0x55b0a0727720): Non uniform sampling or missing slices detected,  maximum nonuniformity:223.316

ImageSeriesReader (0x55b0a0727720): Non uniform sampling or missing slices detected,  maximum nonuniformity:212.073

ImageSeriesReader (0x55b09ffa5780): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000796794

ImageSeriesReader (0x55b09ffa5780): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.00074287

ImageSeriesReader (0x55b09ffa5780): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000800225

ImageSeriesReader (0x55b09ffa5780): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.00078645

ImageSeriesReader (0x55b09ffa5780): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000901594

ImageSeriesReader (0x55b09ffa5780): Non un

Total 50.0 icc data


In [33]:
dficc.head(10)

,pid,pid1,label,RD,img_type,image,mask,series,image_size,mask_size,size_consis
134,1,1307903,2,ICC,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(512, 512, 26)","(512, 512, 26)",True
133,1,1307903,2,ICC,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,3,"(256, 256, 26)","(256, 256, 26)",True
132,1,1307903,2,ICC,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(512, 512, 168)","(512, 512, 168)",True
84,2,1308524,2,ICC,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(512, 512, 22)","(512, 512, 22)",True
85,2,1308524,2,ICC,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,3,"(256, 256, 22)","(256, 256, 22)",True
86,2,1308524,2,ICC,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(512, 512, 92)","(512, 512, 92)",True
100,3,1316577,2,ICC,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(512, 512, 30)","(512, 512, 30)",True
101,3,1316577,2,ICC,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,3,"(256, 256, 28)","(256, 256, 28)",True
99,3,1316577,2,ICC,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(512, 512, 96)","(512, 512, 96)",True
124,10,1340503,2,ICC,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(512, 512, 22)","(512, 512, 22)",True


In [34]:
neg_data = pair_image_mask(root, neg_dir_list, label=0)
pos_data = pair_image_mask(root, pos_dir_list, label=1)
dfn = pd.DataFrame(neg_data, columns=col_names, index=None)
dfp = pd.DataFrame(pos_data, columns=col_names, index=None)
dfn[['pid']] = dfn[['pid']].astype(int)
dfp[['pid']] = dfp[['pid']].astype(int)
dfn = dfn.sort_values(by=['pid','series'], ascending=(True, True))
dfp = dfp.sort_values(by=['pid','series'], ascending=(True, True))
dfall = pd.concat([dfn, dfp], axis=0) 

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000946219

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000744698

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000871834

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000770197

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000796794

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.00074287

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000793774

ImageSeriesReader (0x55b0a05d2e20): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000887337

ImageSeriesReader (0x55b0a05d2e20

In [36]:
dfall.to_csv(f'./{outdata_dir}/all_data_info.csv', index=0)
print('Total {} data'.format(dfall.shape[0]))
dfall.tail(10)

Total 684 data


,pid,pid1,label,RD,img_type,image,mask,series,image_size,mask_size,size_consis
180,229,2829424,1,R2,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(432, 432, 130)","(432, 432, 130)",True
264,230,2833132,1,R2,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(512, 512, 31)","(512, 512, 31)",True
265,230,2833132,1,R2,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,3,"(256, 256, 40)","(256, 256, 40)",True
266,230,2833132,1,R2,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(512, 512, 56)","(512, 512, 56)",True
379,231,2835522,1,R2,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(320, 320, 28)","(320, 320, 28)",True
380,231,2835522,1,R2,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,3,"(256, 128, 26)","(256, 128, 26)",True
378,231,2835522,1,R2,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(320, 260, 72)","(320, 260, 72)",True
79,233,2850457,1,R1,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(512, 512, 46)","(512, 512, 46)",True
80,233,2850457,1,R1,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,3,"(256, 256, 31)","(256, 256, 31)",True
78,233,2850457,1,R1,nii,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(512, 512, 112)","(512, 512, 112)",True


In [53]:
external_dir = '/media/tx-deepocean/Data/2022/chongfu1/Dataset/20220331/external_test'
ex_neg_data = ex_pair_image_mask(external_dir, neg_dir_list, label=0)
ex_pos_data = ex_pair_image_mask(external_dir, pos_dir_list, label=1)
ex_dfn = pd.DataFrame(ex_neg_data, columns=col_names, index=None)
ex_dfp = pd.DataFrame(ex_pos_data, columns=col_names, index=None)
# ex_dfn[['pid']] = ex_dfn[['pid']].astype(int)
# ex_dfp[['pid']] = ex_dfp[['pid']].astype(int)
ex_dfn = ex_dfn.sort_values(by=['pid','series'], ascending=(True, True))
ex_dfp = ex_dfp.sort_values(by=['pid','series'], ascending=(True, True))
ex_dfall = pd.concat([ex_dfn, ex_dfp], axis=0) 

In [54]:
ex_dfall.to_csv(f'./{outdata_dir}/external_data_info.csv', index=0)
print('Total {} data'.format(ex_dfall.shape[0]))
ex_dfall.head(10)

Total 90 data


,pid,pid1,label,RD,img_type,image,mask,series,image_size,mask_size,size_consis
22,12,000050447,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(256,256,30)","(256,256,30)",True
23,12,000050447,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(256,256,30)","(256,256,30)",True
25,16,206583982,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(256,256,30)","(256,256,30)",True
24,16,206583982,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(256,256,30)","(256,256,30)",True
36,17,9301098031,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(256,256,30)","(256,256,30)",True
37,17,9301098031,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(256,256,30)","(256,256,30)",True
15,2,000688543,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(256,256,30)","(256,256,30)",True
14,2,000688543,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(256,256,30)","(256,256,30)",True
20,24,206673717,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,2,"(256,256,30)","(256,256,30)",True
21,24,206673717,0,R0,dicom,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,/media/tx-deepocean/Data/2022/chongfu1/Dataset...,4,"(256,256,30)","(256,256,30)",True
